# <img src="https://img.icons8.com/dusk/64/000000/artificial-intelligence.png" style="height:50px;display:inline"> EE 046202 - Technion - Unsupervised Learning & Data Analysis
---

#### <a href="https://lioritan.github.io">Lior Friedman</a>

## Tutorial 08 - Contrastive Learning - SimCLR

### <img src="https://img.icons8.com/bubbles/50/000000/checklist.png" style="height:50px;display:inline"> Agenda
---
* [Self-supervised Learning in general]
* [Contrastive Learning](#-fff)
    * [Loss Functions](#-fff)
    * [InfoNCE]
    * [NT-Xent]
* [InfoNCE theory - ELBO-style bound](#-elb)
* [SimCLR](#-d)
    * [Idea](#-s)
    * [Implementation - lightly](#-code)
    * [Demo](#-dd)
* [Other contrastive methods]
    * [MoCo](#-df)
    * [BYOL](#-d)
    * [CLIP](#-yo)
* [Recommended Videos](#-Recommended-Videos)
* [Credits](#-Credits)

In [1]:
# imports for the tutorial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib notebook